In [26]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('/content/Task 3 and 4_Loan_Data.csv')

print("="*80)
print("FICO SCORE QUANTIZATION - CREDIT RATING MAP")
print("="*80)
print(f"Dataset: {len(df)} borrowers")
print(f"FICO Range: {df['fico_score'].min()} - {df['fico_score'].max()}")
print(f"Default Rate: {df['default'].mean():.2%}\n")

FICO SCORE QUANTIZATION - CREDIT RATING MAP
Dataset: 10000 borrowers
FICO Range: 408 - 850
Default Rate: 18.51%



In [27]:
def mse_quantization(fico_scores, num_buckets):
    """
    Quantization minimizing Mean Squared Error using equal-frequency buckets.
    """
    bucket_labels = pd.qcut(fico_scores, q=num_buckets, labels=False, duplicates='drop')
    temp_df = pd.DataFrame({'fico': fico_scores, 'bucket': bucket_labels})

    stats = temp_df.groupby('bucket').agg(
        fico_min=('fico', 'min'),
        fico_max=('fico', 'max'),
        fico_mean=('fico', 'mean'),
        count=('fico', 'count')
    ).reset_index()

    # Calculate MSE for each bucket
    mse_values = []
    for bucket_id in stats['bucket']:
        bucket_scores = temp_df[temp_df['bucket'] == bucket_id]['fico'].values
        bucket_mean = stats[stats['bucket'] == bucket_id]['fico_mean'].values[0]
        mse = np.sum((bucket_scores - bucket_mean) ** 2)
        mse_values.append(mse)

    stats['mse'] = mse_values
    total_mse = sum(mse_values)

    # Assign ratings (higher rating = worse credit = lower FICO)
    actual_buckets = len(stats)
    stats['rating'] = actual_buckets - stats['bucket']

    return stats, total_mse

In [29]:
def log_likelihood_quantization(fico_scores, defaults, num_buckets):
    """
    Quantization maximizing log-likelihood considering default rates.
    """
    bucket_labels = pd.qcut(fico_scores, q=num_buckets, labels=False, duplicates='drop')
    temp_df = pd.DataFrame({
        'fico': fico_scores,
        'default': defaults,
        'bucket': bucket_labels
    })

    stats = temp_df.groupby('bucket').agg(
        fico_min=('fico', 'min'),
        fico_max=('fico', 'max'),
        fico_mean=('fico', 'mean'),
        count=('fico', 'count'),
        defaults=('default', 'sum'),
        default_rate=('default', 'mean')
    ).reset_index()

    # Calculate log-likelihood for each bucket
    epsilon = 1e-10
    ll_values = []

    for _, row in stats.iterrows():
        n = row['count']
        k = row['defaults']
        p = np.clip(row['default_rate'], epsilon, 1 - epsilon)
        ll = k * np.log(p) + (n - k) * np.log(1 - p)
        ll_values.append(ll)

    stats['log_likelihood'] = ll_values
    total_ll = sum(ll_values)

    # Assign ratings
    actual_buckets = len(stats)
    stats['rating'] = actual_buckets - stats['bucket']

    return stats, total_ll

In [30]:
print("="*80)
print("METHOD 1: MEAN SQUARED ERROR (MSE) MINIMIZATION")
print("="*80)

for num_buckets in [5, 7, 10]:
    print(f"\n{num_buckets} BUCKETS:")
    print("-"*80)

    stats, total_mse = mse_quantization(df['fico_score'].values, num_buckets)
    print(f"Total MSE: {total_mse:,.0f}\n")
    print(f"{'Rating':<8}{'FICO Range':<20}{'Mean':<12}{'Count':<10}")
    print("-"*50)

    for _, row in stats.iterrows():
        print(f"{int(row['rating']):<8}[{int(row['fico_min'])}, {int(row['fico_max'])}]{' ':<6}"
              f"{row['fico_mean']:<12.1f}{int(row['count']):<10}")


print("\n" + "="*80)
print("METHOD 2: LOG-LIKELIHOOD MAXIMIZATION (Default-Aware)")
print("="*80)

for num_buckets in [5, 7, 10]:
    print(f"\n{num_buckets} BUCKETS:")
    print("-"*80)

    stats, total_ll = log_likelihood_quantization(
        df['fico_score'].values,
        df['default'].values,
        num_buckets
    )

    print(f"Total Log-Likelihood: {total_ll:,.2f}\n")
    print(f"{'Rating':<8}{'FICO Range':<20}{'Count':<8}{'Defaults':<10}{'Def Rate':<12}")
    print("-"*58)

    for _, row in stats.iterrows():
        print(f"{int(row['rating']):<8}[{int(row['fico_min'])}, {int(row['fico_max'])}]{' ':<6}"
              f"{int(row['count']):<8}{int(row['defaults']):<10}{row['default_rate']:<12.4f}")


METHOD 1: MEAN SQUARED ERROR (MSE) MINIMIZATION

5 BUCKETS:
--------------------------------------------------------------------------------
Total MSE: 3,926,495

Rating  FICO Range          Mean        Count     
--------------------------------------------------
5       [408, 587]      552.6       2050      
4       [588, 623]      606.8       1971      
3       [624, 653]      638.6       1989      
2       [654, 688]      670.3       1997      
1       [689, 850]      721.5       1993      

7 BUCKETS:
--------------------------------------------------------------------------------
Total MSE: 2,481,980

Rating  FICO Range          Mean        Count     
--------------------------------------------------
7       [408, 573]      540.6       1442      
6       [574, 604]      589.9       1440      
5       [605, 628]      616.9       1467      
4       [629, 649]      639.0       1385      
3       [650, 672]      660.7       1430      
2       [673, 702]      686.4       1416      
1

In [31]:
print("\n" + "="*80)
print("FINAL RATING MAP - 7 BUCKETS (RECOMMENDED)")
print("="*80)

num_buckets = 7
ll_stats, total_ll = log_likelihood_quantization(
    df['fico_score'].values,
    df['default'].values,
    num_buckets
)

# Create final rating map
rating_map = pd.DataFrame({
    'Rating': ll_stats['rating'].astype(int),
    'FICO_Min': ll_stats['fico_min'].astype(int),
    'FICO_Max': ll_stats['fico_max'].astype(int),
    'FICO_Mean': ll_stats['fico_mean'].round(1),
    'Count': ll_stats['count'].astype(int),
    'Defaults': ll_stats['defaults'].astype(int),
    'Default_Rate': ll_stats['default_rate'].round(4),
    'Risk_Category': ['Very Low Risk', 'Low Risk', 'Moderate-Low Risk',
                      'Moderate Risk', 'Moderate-High Risk', 'High Risk', 'Very High Risk'][::-1]
})

print("\n" + rating_map.to_string(index=False))

# Save to CSV
rating_map.to_csv('fico_rating_map.csv', index=False)
print("\n✓ Rating map saved to: fico_rating_map.csv")


FINAL RATING MAP - 7 BUCKETS (RECOMMENDED)

 Rating  FICO_Min  FICO_Max  FICO_Mean  Count  Defaults  Default_Rate      Risk_Category
      7       408       573      540.6   1442       644        0.4466     Very High Risk
      6       574       604      589.9   1440       370        0.2569          High Risk
      5       605       628      616.9   1467       278        0.1895 Moderate-High Risk
      4       629       649      639.0   1385       226        0.1632      Moderate Risk
      3       650       672      660.7   1430       152        0.1063  Moderate-Low Risk
      2       673       702      686.4   1416       119        0.0840           Low Risk
      1       703       850      732.2   1420        62        0.0437      Very Low Risk

✓ Rating map saved to: fico_rating_map.csv


In [32]:
def map_fico_to_rating(fico_score, rating_map_df):
    """
    Map a FICO score to its corresponding rating.

    Args:
        fico_score: FICO score to map
        rating_map_df: DataFrame with rating boundaries

    Returns:
        rating, risk_category
    """
    for _, row in rating_map_df.iterrows():
        if row['FICO_Min'] <= fico_score <= row['FICO_Max']:
            return row['Rating'], row['Risk_Category']
    return None, 'Unknown'


# Example usage
print("\n" + "="*80)
print("EXAMPLE: Mapping FICO Scores to Ratings")
print("="*80)

test_scores = [450, 550, 600, 650, 700, 750, 800]
print(f"\n{'FICO Score':<15}{'Rating':<10}{'Risk Category':<20}")
print("-"*45)

for score in test_scores:
    rating, risk = map_fico_to_rating(score, rating_map)
    print(f"{score:<15}{rating:<10}{risk:<20}")

print("\n" + "="*80)


EXAMPLE: Mapping FICO Scores to Ratings

FICO Score     Rating    Risk Category       
---------------------------------------------
450            7         Very High Risk      
550            7         Very High Risk      
600            6         High Risk           
650            3         Moderate-Low Risk   
700            2         Low Risk            
750            1         Very Low Risk       
800            1         Very Low Risk       

